# `swift_too` module

## Swift_QueryJob example - Querying and fetching the results of an existing job

### API version = 1.2, `swifttools` version 2.4

#### Author: Jamie A. Kennea (Penn State)

If you wish to retrieve the results of a previously job, this is the task of the QueryJob class. All you need for this class is your username, shared secret and the job number which is reported back when the job is submitted either with `submit()` or `queue()` methods. The jobnumber can be found in the property `jobnumber`.

In [1]:
from swifttools.swift_too import QueryJob, VisQuery
from time import sleep

Previously we would have defined here our username and shared_secret, however in `swifttools 2.2`, for most requests you can skip this and they will default to an anonymous user. For this notebook we'll use the anonymous login to make things simpler.

First let's submit a simple job so we can get an example jobnumber. In this case I'm just going to do something that isn't too strenuous on the system, let's see if Sgr A* is currently visible, or when it is next visible.

In [15]:
vq = VisQuery()
vq.name = "SMC X-2"
vq.hires = (True,)
vq.length = 1
if vq.queue():
    print(f"Queued job successfully! Jobnumber = {vq.status.jobnumber}")
else:
    print(f"Error: {vq.status.errors}")

Queued job successfully! Jobnumber = 466318


Note that the job has been submitted, but because we used the `queue()` method, it may not have completed yet. If you wait a bit, it will be, but the next step can be run straight away to demonstrate what happens if processing isn't complete.



OK let's see how our job is doing, because the purpose of the QueryJob class is to allow us to fetch a job later, or maybe ASAP. Maybe you don't want to hang around waiting for a bunch of jobs to complete, submit some, have some coffee, come back and fetch, or just want to query the result of an existing job.

In [16]:
qj = QueryJob()
qj.jobnumber = vq.status.jobnumber
qj.queue()

print(f"Current status of job #{vq.status.jobnumber}: {qj.status}")

Current status of job #466318: Accepted


Note that for the class `QueryJob`, as it does not take many parameters, you can also pass parameters as arguments. If you do this, then the request will be submitted on invokation, so there's no need use the `submit()` method.

In [17]:
while not qj:
    qj = QueryJob(jobnumber=vq.status.jobnumber)
    print(f"Current status of job #{vq.status.jobnumber}: {qj.status}")
    sleep(1)

The `submit` (or argument form) should return `True` if the job has been fetched. It'll return `False` for a number of reasons, firstly if the job is still *Queued* (waiting to be processed) or *Processing*, the status will indicate this. Also the status can be *Rejected*, in the case where you have given bad parameters, or the job no longer exists.


The above "while" will keep polling the job until it is complete. We put in the sleep between requests so as to not over spam the TOO_API server.

OK let's assume that the above now reports "Current status of job #XXX: Accepted", which indicates a successful `QueryJob` call of a completed job. Let's see what our QueryJob class data now looks like.

In [18]:
qj

username,anonymous
jobnumber,466318
fetchresult,True
status,Accepted
errors,[]
warnings,[]
timestamp,2022-07-15 15:24:21
began,2022-07-15 15:24:21
completed,2022-07-15 15:24:22
result,Swift_VisQuery object


You can see the data inside the class here presented in table form. The the more useful value here is `result`, which contains the result of the job. In the Python api, this will be a `Swift_VisQuery` object. You can display the result of the job like this:

In [19]:
qj.result

2022-07-15 16:16:00,2022-07-15 16:53:00,0:37:00
2022-07-15 17:51:00,2022-07-15 18:29:00,0:38:00
2022-07-15 19:27:00,2022-07-15 20:04:00,0:37:00
2022-07-15 21:02:00,2022-07-15 21:40:00,0:38:00
2022-07-15 22:37:00,2022-07-15 23:15:00,0:38:00
2022-07-16 00:13:00,2022-07-16 00:50:00,0:37:00
2022-07-16 01:58:00,2022-07-16 02:26:00,0:28:00
2022-07-16 03:43:00,2022-07-16 04:01:00,0:18:00
2022-07-16 05:24:00,2022-07-16 05:37:00,0:13:00
2022-07-16 06:34:00,2022-07-16 06:37:00,0:03:00
2022-07-16 07:04:00,2022-07-16 07:12:00,0:08:00


Other parameters that might be of interest are the `timestamp`, `began` and `completed` values, which allow you to find out when the job was submitted, when it started processing and when it finished. For example:

In [20]:
print(
    f"Job was submitted at {qj.timestamp}, was queued for {(qj.began - qj.timestamp).seconds}s and took {(qj.completed - qj.began).seconds}s to process."
)

Job was submitted at 2022-07-15 15:24:21, was queued for 0s and took 1s to process.


As usual, times reported back are UT. That is it! If all is correct, then the following show the upcoming or current *Swift* visibility window for Sgr A*!

In [21]:
_ = [print(win) for win in qj.result]

2022-07-15 16:16:00 - 2022-07-15 16:53:00 (0:37:00)
2022-07-15 17:51:00 - 2022-07-15 18:29:00 (0:38:00)
2022-07-15 19:27:00 - 2022-07-15 20:04:00 (0:37:00)
2022-07-15 21:02:00 - 2022-07-15 21:40:00 (0:38:00)
2022-07-15 22:37:00 - 2022-07-15 23:15:00 (0:38:00)
2022-07-16 00:13:00 - 2022-07-16 00:50:00 (0:37:00)
2022-07-16 01:58:00 - 2022-07-16 02:26:00 (0:28:00)
2022-07-16 03:43:00 - 2022-07-16 04:01:00 (0:18:00)
2022-07-16 05:24:00 - 2022-07-16 05:37:00 (0:13:00)
2022-07-16 06:34:00 - 2022-07-16 06:37:00 (0:03:00)
2022-07-16 07:04:00 - 2022-07-16 07:12:00 (0:08:00)
2022-07-16 08:10:00 - 2022-07-16 08:17:00 (0:07:00)
2022-07-16 08:44:00 - 2022-07-16 08:48:00 (0:04:00)
2022-07-16 09:45:00 - 2022-07-16 09:59:00 (0:14:00)
2022-07-16 11:21:00 - 2022-07-16 11:41:00 (0:20:00)
2022-07-16 12:56:00 - 2022-07-16 13:23:00 (0:27:00)
2022-07-16 14:32:00 - 2022-07-16 15:09:00 (0:37:00)


That first date is going to be the first visibility date, which if the target is visible now should be essentially the same as `timestamp` in `QueryJob`, or sometime in the future if the target was constrained when the job was submitted. 

In [9]:
print(qj.result[0][0] - qj.timestamp)

0:38:37


Note due to the weird way datetime displays negative timediffs, you might see something like `-1 day, 23:59:19` if the value is negative!